In [28]:
import pandas as pd
import json

# Định nghĩa nhãn các checkbox theo thứ tự xuất hiện trong HTML
INFRA_LABELS = [
    "Được đầu tư mua trụ sở/ thuê địa điểm mới",
    "Áp dụng mô hình sảnh giao dịch chuẩn",
    "Được trang bị máy R-ATM",
    "Được trang bị hệ thống lấy số tự động"
]

POLICY_LABELS = [
    "Nâng mức thẩm quyền cấp tín dụng",
    "Nâng mức thẩm quyền giải ngân",
    "Nâng hạn mức phê duyệt giao dịch",
    "Mở rộng đối tượng phân khúc KHDN",
    "Có các sản phẩm đặc thù dành riêng cho Siêu PGD"
]

SUPPORT_LABELS = [
    "Cơ chế hỗ trợ chi phí xây dựng, sửa chữa, cải tạo trụ sở",
    "Ưu tiên hỗ trợ từ TSC/CN về nguồn khách hàng/đối tác",
    "Bổ sung thêm nhân sự cho CN",
    "Cơ chế động lực (lương, thưởng)",
    "Các chương trình đào tạo chuyên sâu"
]

def tach_du_lieu_survey(json_string):
    try:
        data = json.loads(json_string)
    except Exception as e:
        print("Lỗi parse JSON:", e)
        return None

    # Lấy danh sách checkbox đã chọn
    infra_checked = data.get("infrastructure", [])
    policy_checked = data.get("policy", [])
    support_checked = data.get("support", [])

    # Lấy danh sách các ô "ý kiến bổ sung"
    supp = data.get("infrastructure_supplementary", [])
    # Đảm bảo mảng supp có đủ 14 phần tử (4 cho infra, 5 cho policy, 5 cho support)
    while len(supp) < 14:
        supp.append("")

    ket_qua = {}

    # ---------- Phần Infrastructure (9 cột) ----------
    for i, label in enumerate(INFRA_LABELS):
        cot_check = f"infra_{i+1}_check"
        cot_text  = f"infra_{i+1}_text"
        ket_qua[cot_check] = "Yes" if label in infra_checked else "No"
        ket_qua[cot_text]  = supp[i]   # chỉ số 0 đến 3
    ket_qua["infra_others"] = data.get("infrastructure_others", "")

    # ---------- Phần Policy (11 cột) ----------
    for i, label in enumerate(POLICY_LABELS):
        cot_check = f"policy_{i+1}_check"
        cot_text  = f"policy_{i+1}_text"
        ket_qua[cot_check] = "Yes" if label in policy_checked else "No"
        # Các ô text của Policy nằm ở supp[4] đến supp[8]
        ket_qua[cot_text]  = supp[4 + i]
    ket_qua["policy_others"] = data.get("policy_others", "")

    # ---------- Phần Support (11 cột) ----------
    for i, label in enumerate(SUPPORT_LABELS):
        cot_check = f"support_{i+1}_check"
        cot_text  = f"support_{i+1}_text"
        ket_qua[cot_check] = "Yes" if label in support_checked else "No"
        # Các ô text của Support nằm ở supp[9] đến supp[13]
        ket_qua[cot_text]  = supp[9 + i]
    ket_qua["support_others"] = data.get("support_others", "")

    # ---------- Phần các câu hỏi khác (5 cột) ----------
    ket_qua["propose_sieupgd"] = data.get("propose_sieupgd", "")
    ket_qua["pgd_code"] = data.get("pgd_code", "")
    ket_qua["pgd_name"] = data.get("pgd_name", "")
    ket_qua["other_banks_info"] = data.get("other_banks_info", "")
    ket_qua["additional_opinions"] = data.get("additional_opinions", "")

    # ---------- Thêm cột bổ sung: user_id (lấy từ cột "User" của file Excel) ----------
    ket_qua["user_id"] = ""

    return ket_qua

def main():
    # Đọc file Excel ban đầu; file có các cột: "User", "Response"
    df = pd.read_excel("/home/trungdt2/Downloads/overall_report.xlsx")
    print("Các cột có trong file:", df.columns.tolist())
    
    ds_ban_ghi = []
    # Sử dụng cột "Response" để parse JSON
    for index, row in df.iterrows():
        json_string = row.get("Response")  # cột chứa JSON
        if pd.isna(json_string) or json_string == "":
            continue
        ban_ghi = tach_du_lieu_survey(json_string)
        if ban_ghi is None:
            continue
        # Gán user_id từ cột "User"
        ban_ghi["user_id"] = row.get("User", "")
        ds_ban_ghi.append(ban_ghi)
    
    if not ds_ban_ghi:
        print("Không có bản ghi nào được parse thành công.")
        return
    
    df_moi = pd.DataFrame(ds_ban_ghi)
    print("Các cột mới:", df_moi.columns.tolist())
    df_moi.to_excel("/home/trungdt2/Downloads/survey_parsed.xlsx", index=False)
    print("Xuất file survey_parsed.xlsx thành công.")

if __name__ == "__main__":
    main()


Các cột có trong file: ['User', 'Response']
Các cột mới: ['infra_1_check', 'infra_1_text', 'infra_2_check', 'infra_2_text', 'infra_3_check', 'infra_3_text', 'infra_4_check', 'infra_4_text', 'infra_others', 'policy_1_check', 'policy_1_text', 'policy_2_check', 'policy_2_text', 'policy_3_check', 'policy_3_text', 'policy_4_check', 'policy_4_text', 'policy_5_check', 'policy_5_text', 'policy_others', 'support_1_check', 'support_1_text', 'support_2_check', 'support_2_text', 'support_3_check', 'support_3_text', 'support_4_check', 'support_4_text', 'support_5_check', 'support_5_text', 'support_others', 'propose_sieupgd', 'pgd_code', 'pgd_name', 'other_banks_info', 'additional_opinions', 'user_id']
Xuất file survey_parsed.xlsx thành công.


In [26]:
df.head()

,User,Response
0,CHAUNTK,"{""infrastructure"": [""Được đầu tư mua trụ sở/ t..."
1,SONTA,"{""infrastructure"": [""Được đầu tư mua trụ sở/ t..."


In [32]:
import pandas as pd
import json

# Định nghĩa nhãn các checkbox theo thứ tự xuất hiện trong HTML
NHAN_INFRA = [
    "Được đầu tư mua trụ sở/ thuê địa điểm mới",
    "Áp dụng mô hình sảnh giao dịch chuẩn",
    "Được trang bị máy R-ATM",
    "Được trang bị hệ thống lấy số tự động"
]

NHAN_POLICY = [
    "Nâng mức thẩm quyền cấp tín dụng",
    "Nâng mức thẩm quyền giải ngân",
    "Nâng hạn mức phê duyệt giao dịch",
    "Mở rộng đối tượng phân khúc KHDN",
    "Có các sản phẩm đặc thù dành riêng cho Siêu PGD"
]

NHAN_SUPPORT = [
    "Cơ chế hỗ trợ chi phí xây dựng, sửa chữa, cải tạo trụ sở",
    "Ưu tiên hỗ trợ từ TSC/CN về nguồn khách hàng/đối tác",
    "Bổ sung thêm nhân sự cho CN",
    "Cơ chế động lực (lương, thưởng)",
    "Các chương trình đào tạo chuyên sâu"
]

def tach_du_lieu_survey(json_string):
    try:
        data = json.loads(json_string)
    except Exception as e:
        print("Lỗi parse JSON:", e)
        return None

    # Lấy danh sách checkbox đã chọn
    infra_checked = data.get("infrastructure", [])
    policy_checked = data.get("policy", [])
    support_checked = data.get("support", [])

    # Lấy danh sách các ô "ý kiến bổ sung"
    supp = data.get("infrastructure_supplementary", [])
    # Đảm bảo mảng supp có đủ 14 phần tử (4 cho cơ sở, 5 cho chính sách, 5 cho hỗ trợ)
    while len(supp) < 14:
        supp.append("")

    ket_qua = {}

    # ---------- Phần Cơ Sở Vật Chất (9 cột) ----------
    # Đặt tên cột: "Cơ sở 1: Chọn", "Cơ sở 1: Ý kiến bổ sung", ...
    for i, nhan in enumerate(NHAN_INFRA):
        cot_check = f"Co_so_{i+1}_Chon"
        cot_text  = f"Co_so_{i+1}_YKien"
        ket_qua[cot_check] = "Có" if nhan in infra_checked else "Không"
        ket_qua[cot_text]  = supp[i]
    ket_qua["Co_so_khac"] = data.get("infrastructure_others", "")

    # ---------- Phần Chính Sách (11 cột) ----------
    for i, nhan in enumerate(NHAN_POLICY):
        cot_check = f"Chinh_sach_{i+1}_Chon"
        cot_text  = f"Chinh_sach_{i+1}_YKien"
        ket_qua[cot_check] = "Có" if nhan in policy_checked else "Không"
        # Các ô text của Chính sách nằm ở supp[4] đến supp[8]
        ket_qua[cot_text]  = supp[4 + i]
    ket_qua["Chinh_sach_khac"] = data.get("policy_others", "")

    # ---------- Phần Hỗ Trợ (11 cột) ----------
    for i, nhan in enumerate(NHAN_SUPPORT):
        cot_check = f"Ho_tro_{i+1}_Chon"
        cot_text  = f"Ho_tro_{i+1}_YKien"
        ket_qua[cot_check] = "Có" if nhan in support_checked else "Không"
        # Các ô text của Hỗ trợ nằm ở supp[9] đến supp[13]
        ket_qua[cot_text]  = supp[9 + i]
    ket_qua["Ho_tro_khac"] = data.get("support_others", "")

    # ---------- Các Câu Hỏi Khác (5 cột) ----------
    ket_qua["De_xuat"] = data.get("propose_sieupgd", "")
    ket_qua["Ma_PGD"] = data.get("pgd_code", "")
    ket_qua["Ten_PGD"] = data.get("pgd_name", "")
    ket_qua["Thong_tin_PGD_khac"] = data.get("other_banks_info", "")
    ket_qua["YKien_khac"] = data.get("additional_opinions", "")

    # ---------- Thêm cột bổ sung: user_id ----------
    ket_qua["User"] = ""  # Sẽ được gán từ cột "User" trong file Excel

    return ket_qua

def main():
    # Đọc file Excel ban đầu (các cột: "User", "Response")
    df = pd.read_excel("/home/trungdt2/Downloads/overall_report.xlsx")
    print("Các cột có trong file:", df.columns.tolist())
    
    ds_ban_ghi = []
    # Dùng cột "Response" để parse JSON
    for index, row in df.iterrows():
        json_string = row.get("Response")
        if pd.isna(json_string) or json_string == "":
            continue
        ban_ghi = tach_du_lieu_survey(json_string)
        if ban_ghi is None:
            continue
        # Gán giá trị từ cột "User"
        ban_ghi["User"] = row.get("User", "")
        ds_ban_ghi.append(ban_ghi)
    
    if not ds_ban_ghi:
        print("Không có bản ghi nào được parse thành công.")
        return
    
    df_moi = pd.DataFrame(ds_ban_ghi)
    print("Các cột mới:", df_moi.columns.tolist())
    df_moi.to_excel("/home/trungdt2/Downloads/survey_parsed2.xlsx", index=False)
    print("Xuất file survey_parsed.xlsx thành công.")

if __name__ == "__main__":
    main()


Các cột có trong file: ['User', 'Response']
Các cột mới: ['Co_so_1_Chon', 'Co_so_1_YKien', 'Co_so_2_Chon', 'Co_so_2_YKien', 'Co_so_3_Chon', 'Co_so_3_YKien', 'Co_so_4_Chon', 'Co_so_4_YKien', 'Co_so_khac', 'Chinh_sach_1_Chon', 'Chinh_sach_1_YKien', 'Chinh_sach_2_Chon', 'Chinh_sach_2_YKien', 'Chinh_sach_3_Chon', 'Chinh_sach_3_YKien', 'Chinh_sach_4_Chon', 'Chinh_sach_4_YKien', 'Chinh_sach_5_Chon', 'Chinh_sach_5_YKien', 'Chinh_sach_khac', 'Ho_tro_1_Chon', 'Ho_tro_1_YKien', 'Ho_tro_2_Chon', 'Ho_tro_2_YKien', 'Ho_tro_3_Chon', 'Ho_tro_3_YKien', 'Ho_tro_4_Chon', 'Ho_tro_4_YKien', 'Ho_tro_5_Chon', 'Ho_tro_5_YKien', 'Ho_tro_khac', 'De_xuat', 'Ma_PGD', 'Ten_PGD', 'Thong_tin_PGD_khac', 'YKien_khac', 'User']
Xuất file survey_parsed.xlsx thành công.
